# MERA pathology


In [3]:
#### import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import Sequence
import pandas as pd
import os 
tf.compat.v1.enable_v2_behavior
# Import tensornetwork
import tensornetwork as tn
# Set the backend to tesorflow
# (default is numpy)
tn.set_default_backend("tensorflow")
batch_size = 128
learning_rate = 0.001


class LIDC(Sequence):
    def __init__(self, rater=4, split='Train', data_dir = './', transform=None):
        super().__init__()

        self.data_dir = data_dir
        self.rater = rater
        self.transform = transform
        self.data, self.targets = torch.load(data_dir+split+'.pt')
        self.targets = self.targets.type(torch.FloatTensor)		   
    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):

        image, label = self.data[index], self.targets[index]
        if self.rater == 4:
            label = (label.sum() > 2).type_as(self.targets)
        else:
            label = label[self.rater]
        image = image.type(torch.FloatTensor)/255.0
        if self.transform is not None:
            image = self.transform(image)
        return image.numpy(), label.numpy()

    

class Batcher(Sequence):
    """Assemble a sequence of things into a sequence of batches."""
    def __init__(self, sequence, batch_size=16):
        self._batch_size = batch_size
        self._sequence = sequence
        self._idxs = np.arange(len(self._sequence))

    def __len__(self):
        return int(np.ceil(len(self._sequence) / self._batch_size))

    def __getitem__(self, i):
        if i >= len(self):
            raise IndexError("Index out of bounds")

        start = i*self._batch_size
        end = min(len(self._sequence), start+self._batch_size)
        data = [self._sequence[j] for j in self._idxs[start:end]]
        inputs = [d[0] for d in data]
        outputs = [d[1] for d in data]

        return self._stack(inputs), self._stack(outputs)

    def _stack(self, data):
        if data is None:
            return None

        if not isinstance(data[0], (list, tuple)):
            return np.stack(data)

        seq = type(data[0])
        K = len(data[0])
        data = seq(
            np.stack([d[k] for d in data])
            for k in range(K)
        )

        return data

    def on_epoch_end(self):
        np.random.shuffle(self._idxs)
        self._sequence.on_epoch_end()
        
        
        
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
tf.compat.v1.enable_v2_behavior
# Import tensornetwork
import tensornetwork as tn
from tensorflow.keras.utils import to_categorical
tn.set_default_backend("tensorflow")

data_path = '/datacommons/carin/fk43/LIDC/'
train_data=dataset_train = LIDC(split='Train', data_dir=data_path,
                                transform=None,rater=4)
valid_data=dataset_train = LIDC(split='Valid', data_dir=data_path,
                                transform=None,rater=4)
test_data=dataset_train = LIDC(split='Test', data_dir=data_path,
                                transform=None,rater=4)

NameError: name 'torch' is not defined